In [1]:
%pip install ../..

Processing /home/hyang/SpuCo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=81027 sha256=502b1544050eb4edaa2bfccf22bb98bcc478eccda1e66e94d015a1fb05ed3c7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-a43v2a9e/wheels/16/2e/00/5bdefcfd7f850d6f19880ecdb4dd3f325f4b906bf004cd82e3
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np

a = np.load("/home/hyang/SpuCo/quickstart/samples/res/jtt_result_1e-05_60.npz")

In [3]:
a["worst_acc"]

array(33.95638629)

In [ ]:
import torch 

device = torch.device("cuda:7")

In [ ]:
from wilds import get_dataset
import torchvision.transforms as transforms

# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="waterbirds", download=True)

transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

# Get the training set
train_data = dataset.get_subset(
    "train",
    transform=transform
)

# Get the training set
test_data = dataset.get_subset(
    "test",
    transform=transform
)

In [ ]:
infer_lr = 1e-3
infer_momentum = 0.9
infer_epoch = 30
infer_upsample = 100

dro_lr = 1e-3
dro_momenturm = 0.9
dro_epoch = 30
dro_batch_size = 128


In [ ]:
from spuco.utils import WILDSDatasetWrapper

trainset = WILDSDatasetWrapper(dataset=train_data, metadata_spurious_label="background", verbose=True)
testset = WILDSDatasetWrapper(dataset=test_data, metadata_spurious_label="background", verbose=True)

In [ ]:
from spuco.group_inference import JTTInference
from spuco.models import model_factory 
from spuco.utils import Trainer
from torch.optim import SGD

model = model_factory("resnet50", trainset[0][0].shape, trainset.num_classes).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=128,
    optimizer=SGD(model.parameters(), lr=infer_lr, momentum=infer_momentum),
    device=device,
    verbose=True
)
trainer.train(infer_epoch)

In [ ]:
from spuco.utils import get_class_labels 

predictions = torch.argmax(trainer.get_trainset_outputs(), dim=-1).detach().cpu().tolist()
jtt = JTTInference(
    predictions=predictions,
    class_labels=get_class_labels(trainset)
)

In [ ]:
group_partition = jtt.infer_groups()

In [ ]:
for key in group_partition.keys():
    print(key, len(group_partition[key]))

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
from spuco.utils import GroupLabeledDataset

group_labeled_trainset = GroupLabeledDataset(trainset, group_partition)

In [ ]:
upsample_count = infer_upsample
indices = []
for key in group_partition.keys():
    group_reordered_indices = [group_partition[key][i] for i in torch.randperm(len(group_partition[key])).tolist()]
    group_indices = []
    while len(group_indices) < upsample_count:
        group_indices.extend(group_reordered_indices)
        indices.extend(group_indices[:upsample_count])

In [ ]:
from torch.optim import SGD
from spuco.invariant_train import UpSampleERM, DownSampleERM, CustomSampleERM
from spuco.models import model_factory 

model = model_factory("resnet50", trainset[0][0].shape, trainset.num_classes).to(device)

jtt_train = CustomSampleERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    indices=indices,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9),
    device=device,
    verbose=True
)
jtt_train.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy[1]

In [ ]:
evaluator.average_accuracy

In [ ]:
import numpy as np

np.savez("res/jtt_result_{}_{}".format(str(infer_lr), str(infer_epoch)), worst_acc=evaluator.worst_group_accuracy[1], ave_acc=evaluator.average_accuracy)